In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
data = pd.read_csv('~/LIWC_top5companies.csv')
print(data.shape)
data.head()

(14002, 38)


,A,B,C,D,E,F,G,H,I,J,K,L,WC,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak
0,screen_name,created_at,tweet_id,text,extended_tweet,in_reply_to_screen_name,in_reply_to_status_id,retweet_count,favorite_count,truncated,language,in_reply_to_user_id,2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0
1,GoldmanSachs,2018-07-16 20:42:39,1018959118785220608,REMINDER: $GS earnings will be out tomorrow on...,REMINDER: $GS earnings will be out tomorrow on...,NaN,NaN,11,18,NaN,en,NaN,21,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.76,0.0,4.76,0.00,0.0,0.0,0.0,0.0
2,GoldmanSachs,2018-08-02 17:35:25,1025072590182907904,Congratulations to #Apple and @tim_cook on mak...,Congratulations to #Apple and @tim_cook on mak...,NaN,NaN,131,287,NaN,en,NaN,18,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.56,5.56,0.0,0.0,0.0,0.0
3,GoldmanSachs,2018-10-15 20:30:07,1051933258999635968,REMINDER: $GS earnings will be out tomorrow on...,REMINDER: $GS earnings will be out tomorrow on...,NaN,NaN,15,51,NaN,en,NaN,21,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.76,0.0,4.76,0.00,0.0,0.0,0.0,0.0
4,GoldmanSachs,2019-04-12 20:30:04,1116800672341680135,REMINDER: $GS earnings will be out Monday on t...,REMINDER: $GS earnings will be out Monday on t...,NaN,NaN,11,34,NaN,en,NaN,23,4.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.35,0.0,4.35,0.00,0.0,0.0,0.0,0.0


In [ ]:
data = data.rename(columns={'A': 'screen_name',
                            'B': 'created_at',
                            'C': 'tweet_id',
                            'D': 'text',
                            'E': 'extended_tweet',
                            'F': 'in_reply_to_screen_name',
                            'G': 'in_reply_to_status_id',
                            'H': 'retweet_count',
                            'I': 'favorite_count',
                            'J': 'truncated',
                            'K': 'language',
                            'L': 'in_reply_to_user_id'})
data = data.drop(data.index[0])
print(data.shape)
data.head()

(14001, 38)


,screen_name,created_at,tweet_id,text,extended_tweet,in_reply_to_screen_name,in_reply_to_status_id,retweet_count,favorite_count,truncated,language,in_reply_to_user_id,WC,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak
1,GoldmanSachs,2018-07-16 20:42:39,1018959118785220608,REMINDER: $GS earnings will be out tomorrow on...,REMINDER: $GS earnings will be out tomorrow on...,NaN,NaN,11,18,NaN,en,NaN,21,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.76,0.0,4.76,0.00,0.0,0.0,0.0,0.0
2,GoldmanSachs,2018-08-02 17:35:25,1025072590182907904,Congratulations to #Apple and @tim_cook on mak...,Congratulations to #Apple and @tim_cook on mak...,NaN,NaN,131,287,NaN,en,NaN,18,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,5.56,5.56,0.0,0.0,0.0,0.0
3,GoldmanSachs,2018-10-15 20:30:07,1051933258999635968,REMINDER: $GS earnings will be out tomorrow on...,REMINDER: $GS earnings will be out tomorrow on...,NaN,NaN,15,51,NaN,en,NaN,21,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.76,0.0,4.76,0.00,0.0,0.0,0.0,0.0
4,GoldmanSachs,2019-04-12 20:30:04,1116800672341680135,REMINDER: $GS earnings will be out Monday on t...,REMINDER: $GS earnings will be out Monday on t...,NaN,NaN,11,34,NaN,en,NaN,23,4.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.70,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.35,0.0,4.35,0.00,0.0,0.0,0.0,0.0
5,GoldmanSachs,2019-06-05 20:00:09,1136362091802238981,Tune in to @BloombergTV tomorrow morning from ...,Tune in to @BloombergTV tomorrow morning from ...,NaN,NaN,2,7,NaN,en,NaN,21,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.76,0.0,0.0,0.0,0.0,0.0,4.76,0.0,0.00,0.0,9.52,0.00,0.0,0.0,0.0,0.0


In [ ]:
data.dtypes

screen_name                 object
created_at                  object
tweet_id                    object
text                        object
extended_tweet              object
in_reply_to_screen_name     object
in_reply_to_status_id       object
retweet_count               object
favorite_count              object
truncated                   object
language                    object
in_reply_to_user_id         object
WC                           int64
we                         float64
negate                     float64
compare                    float64
affect                     float64
posemo                     float64
negemo                     float64
anx                        float64
anger                      float64
sad                        float64
social                     float64
family                     float64
friend                     float64
female                     float64
male                       float64
certain                    float64
health              

In [ ]:
data['screen_name'].unique()

array(['GoldmanSachs', 'Nasdaq', 'Cognizant', 'salesforce', 'SPGlobal'],
      dtype=object)

# GoldmanSachs

In [ ]:
# subset data for the GoldmanSachs company
GoldmanSachs = data['screen_name'] == 'GoldmanSachs'
GoldmanSachs = data[GoldmanSachs]
# select the datetime, the targeted LIWC output variables, and the integral text data
GoldmanSachs = GoldmanSachs[['created_at','we','negate','compare','affect','posemo','negemo','anx','anger',
                             'sad','social','family','friend','female','male','certain','health','achieve',
                             'reward','risk','work','leisure','home','death','swear','netspeak','extended_tweet']]
# for the datetime column keep the date only
GoldmanSachs['created_at'] = GoldmanSachs['created_at'].str.split(' ').str[0]
# convert the data type
GoldmanSachs.created_at = pd.to_datetime(GoldmanSachs.created_at, format='%Y-%m-%d')
# aggregate all columns at days except the text data concatenated but separated by '||'
GoldmanSachs = GoldmanSachs.groupby('created_at').agg(we=pd.NamedAgg(column='we', aggfunc=sum),
                                                           negate=pd.NamedAgg(column='negate', aggfunc=sum),
                                                           compare=pd.NamedAgg(column='compare', aggfunc=sum),
                                                           affect=pd.NamedAgg(column='affect', aggfunc=sum),
                                                           posemo=pd.NamedAgg(column='posemo', aggfunc=sum),
                                                           negemo=pd.NamedAgg(column='negemo', aggfunc=sum),
                                                           anx=pd.NamedAgg(column='anx', aggfunc=sum),
                                                           anger=pd.NamedAgg(column='anger', aggfunc=sum),
                                                           sad=pd.NamedAgg(column='sad', aggfunc=sum),
                                                           social=pd.NamedAgg(column='social', aggfunc=sum),
                                                           family=pd.NamedAgg(column='family', aggfunc=sum),
                                                           friend=pd.NamedAgg(column='friend', aggfunc=sum),
                                                           female=pd.NamedAgg(column='female', aggfunc=sum),
                                                           male=pd.NamedAgg(column='male', aggfunc=sum),
                                                           certain=pd.NamedAgg(column='certain', aggfunc=sum),
                                                           health=pd.NamedAgg(column='health', aggfunc=sum),
                                                           achieve=pd.NamedAgg(column='achieve', aggfunc=sum),
                                                           reward=pd.NamedAgg(column='reward', aggfunc=sum),
                                                           risk=pd.NamedAgg(column='risk', aggfunc=sum),
                                                           work=pd.NamedAgg(column='work', aggfunc=sum),
                                                           leisure=pd.NamedAgg(column='leisure', aggfunc=sum),
                                                           home=pd.NamedAgg(column='home', aggfunc=sum),
                                                           death=pd.NamedAgg(column='death', aggfunc=sum),
                                                           swear=pd.NamedAgg(column='swear', aggfunc=sum),
                                                           netspeak=pd.NamedAgg(column='netspeak', aggfunc=sum),
                                                           text_agg=pd.NamedAgg(column='extended_tweet', aggfunc=lambda x: x.str.cat(sep='||')))
# lowercase the text data
GoldmanSachs['text_agg'] = GoldmanSachs['text_agg'].str.lower()
# create the dummy variable for text data
GoldmanSachs['dummy_key'] = GoldmanSachs['text_agg'].apply(lambda x: 1 if ('covid' in x)|('pandemic' in x)|('coronavirus' in x) else 0)
GoldmanSachs

,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak,text_agg,dummy_key
created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-25,3.57,0.00,12.27,12.68,12.68,0.00,0.0,0.0,0.0,33.83,0.00,0.00,13.05,0.00,0.00,0.00,12.87,0.00,0.00,17.53,19.23,0.00,0.0,0.0,21.70,#threelions roaring? $gs' mathematical model n...,0
2018-06-26,11.54,3.85,8.34,17.44,8.72,4.55,0.0,0.0,0.0,49.17,0.00,0.00,0.00,12.18,3.85,0.00,8.72,4.55,4.55,38.05,16.35,0.00,0.0,0.0,25.14,watch: $gs president &amp; coo david solomon j...,0
2018-06-27,4.76,0.00,0.00,8.20,8.20,0.00,0.0,0.0,0.0,29.76,0.00,0.00,4.35,3.85,0.00,4.35,3.85,3.85,0.00,20.73,4.35,0.00,0.0,0.0,22.07,watch: beyond tariffs—what’s at stake in the u...,0
2018-06-28,4.55,0.00,4.55,13.64,13.64,0.00,0.0,0.0,0.0,17.81,0.00,0.00,4.55,0.00,0.00,0.00,26.73,4.55,0.00,38.69,4.17,0.00,0.0,0.0,28.10,watch: $gs' co-head of asia macro research and...,0
2018-06-29,0.00,3.70,16.57,18.33,18.33,0.00,0.0,0.0,0.0,16.85,4.17,8.33,0.00,0.00,0.00,0.00,4.55,0.00,0.00,36.19,4.17,4.17,0.0,0.0,29.17,view from $gs tokyo office: #summer has arrive...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-05,8.00,0.00,4.00,3.85,3.85,0.00,0.0,0.0,0.0,20.20,0.00,0.00,0.00,0.00,0.00,0.00,8.00,4.00,0.00,16.00,3.85,0.00,0.0,0.0,16.20,to continue making progress in further increas...,0
2020-08-06,10.52,0.00,20.20,34.33,34.33,0.00,0.0,0.0,0.0,47.87,0.00,0.00,0.00,0.00,0.00,0.00,10.67,3.85,0.00,32.17,0.00,0.00,0.0,0.0,36.21,"""part of what makes this moment different is t...",0
2020-08-07,0.00,9.52,0.00,23.05,23.05,0.00,0.0,0.0,0.0,44.12,0.00,0.00,5.88,0.00,4.35,0.00,4.35,4.35,0.00,54.01,0.00,0.00,0.0,0.0,46.94,"""colonialism [is] not being taught in school. ...",0


In [ ]:
# plot the trends
x_axis = GoldmanSachs.index
fig = go.Figure()
annotations = []

for i in range(0,25,1):
  y = GoldmanSachs.iloc[:,i].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[:], y=y,
                    mode='lines',
                    name=GoldmanSachs.columns[i]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LIWC Output for GoldmanSachs',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations, 
                  autosize=False,
                  width=1200,
                  height=500,
                  margin=dict(l=10, r=10, b=50, t=50, pad=4))
fig.show()

# Nasdaq

In [ ]:
# subset data for the Nasdaq company
Nasdaq = data['screen_name'] == 'Nasdaq'
Nasdaq = data[Nasdaq]
# select the datetime, the targeted LIWC output variables, and the integral text data
Nasdaq = Nasdaq[['created_at','we','negate','compare','affect','posemo','negemo','anx','anger',
                             'sad','social','family','friend','female','male','certain','health','achieve',
                             'reward','risk','work','leisure','home','death','swear','netspeak','extended_tweet']]
# for the datetime column keep the date only
Nasdaq['created_at'] = Nasdaq['created_at'].str.split(' ').str[0]
# convert the data type
Nasdaq.created_at = pd.to_datetime(Nasdaq.created_at, format='%Y-%m-%d')
# aggregate all columns at days except the text data concatenated but separated by '||'
Nasdaq = Nasdaq.groupby('created_at').agg(we=pd.NamedAgg(column='we', aggfunc=sum),
                                                           negate=pd.NamedAgg(column='negate', aggfunc=sum),
                                                           compare=pd.NamedAgg(column='compare', aggfunc=sum),
                                                           affect=pd.NamedAgg(column='affect', aggfunc=sum),
                                                           posemo=pd.NamedAgg(column='posemo', aggfunc=sum),
                                                           negemo=pd.NamedAgg(column='negemo', aggfunc=sum),
                                                           anx=pd.NamedAgg(column='anx', aggfunc=sum),
                                                           anger=pd.NamedAgg(column='anger', aggfunc=sum),
                                                           sad=pd.NamedAgg(column='sad', aggfunc=sum),
                                                           social=pd.NamedAgg(column='social', aggfunc=sum),
                                                           family=pd.NamedAgg(column='family', aggfunc=sum),
                                                           friend=pd.NamedAgg(column='friend', aggfunc=sum),
                                                           female=pd.NamedAgg(column='female', aggfunc=sum),
                                                           male=pd.NamedAgg(column='male', aggfunc=sum),
                                                           certain=pd.NamedAgg(column='certain', aggfunc=sum),
                                                           health=pd.NamedAgg(column='health', aggfunc=sum),
                                                           achieve=pd.NamedAgg(column='achieve', aggfunc=sum),
                                                           reward=pd.NamedAgg(column='reward', aggfunc=sum),
                                                           risk=pd.NamedAgg(column='risk', aggfunc=sum),
                                                           work=pd.NamedAgg(column='work', aggfunc=sum),
                                                           leisure=pd.NamedAgg(column='leisure', aggfunc=sum),
                                                           home=pd.NamedAgg(column='home', aggfunc=sum),
                                                           death=pd.NamedAgg(column='death', aggfunc=sum),
                                                           swear=pd.NamedAgg(column='swear', aggfunc=sum),
                                                           netspeak=pd.NamedAgg(column='netspeak', aggfunc=sum),
                                                           text_agg=pd.NamedAgg(column='extended_tweet', aggfunc=lambda x: x.str.cat(sep='||')))
# lowercase the text data
Nasdaq['text_agg'] = Nasdaq['text_agg'].str.lower()
# create the dummy variable for text data
Nasdaq['dummy_key'] = Nasdaq['text_agg'].apply(lambda x: 1 if ('covid' in x)|('pandemic' in x)|('coronavirus' in x) else 0)
Nasdaq

,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak,text_agg,dummy_key
created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-05-28,0.00,8.40,12.24,16.79,4.55,12.24,4.55,0.0,3.85,30.40,0.00,0.0,0.00,4.55,3.85,13.86,22.70,4.55,3.85,49.71,5.00,0.00,0.0,0.0,37.26,🎉congratulations to @angiodynamics which is ce...,0
2019-05-29,0.00,4.35,12.35,8.35,8.35,0.00,0.00,0.0,0.00,9.09,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.83,0.00,0.00,0.0,0.0,12.90,🎧listen: @evanharvey99 and @yale lecturer in s...,0
2019-05-30,4.76,0.00,12.00,30.70,25.94,4.76,4.76,0.0,0.00,67.67,0.00,0.0,0.00,0.00,5.56,13.22,35.11,15.76,14.28,55.48,13.91,4.00,0.0,0.0,40.61,⚡️conmed corporation is innovating surgical te...,0
2019-05-31,4.00,0.00,9.56,5.56,5.56,0.00,0.00,0.0,0.00,9.56,0.00,0.0,0.00,0.00,0.00,5.56,11.11,5.56,4.00,16.70,8.35,0.00,0.0,0.0,17.91,we need to eliminate the order protection rule...,0
2019-06-01,0.00,8.00,0.00,8.00,4.00,4.00,0.00,0.0,4.00,8.55,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.64,0.00,0.00,0.0,0.0,12.55,"1,000+ members of the ir and capital markets e...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-05,19.14,0.00,11.14,37.94,37.94,0.00,0.00,0.0,0.00,94.51,17.18,0.0,0.00,3.57,13.61,0.00,16.76,17.11,4.00,71.69,17.18,17.18,0.0,0.0,50.76,defiance etfs announced the launch of the defi...,0
2020-08-06,7.41,3.70,4.35,20.77,17.07,3.70,0.00,0.0,3.70,50.54,4.55,0.0,0.00,4.00,0.00,3.70,8.00,12.68,4.00,68.44,8.90,4.55,0.0,0.0,34.02,.@nasdaq chief economist @phil_mackintosh disc...,0
2020-08-07,12.35,0.00,12.00,22.46,13.70,8.76,4.76,4.0,0.00,78.53,9.35,0.0,0.00,4.17,13.76,0.00,21.24,26.16,0.00,47.98,22.61,9.35,0.0,0.0,39.76,today! join subject matter experts from nasdaq...,1


In [ ]:
# plot the trends
x_axis = Nasdaq.index
fig = go.Figure()
annotations = []

for i in range(0,25,1):
  y = Nasdaq.iloc[:,i].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[:], y=y,
                    mode='lines',
                    name=Nasdaq.columns[i]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LIWC Output for Nasdaq',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations, 
                  autosize=False,
                  width=1200,
                  height=500,
                  margin=dict(l=10, r=10, b=50, t=50, pad=4))
fig.show()

# Cognizant

In [ ]:
# subset data for the Cognizant company
Cognizant = data['screen_name'] == 'Cognizant'
Cognizant = data[Cognizant]
# select the datetime, the targeted LIWC output variables, and the integral text data
Cognizant = Cognizant[['created_at','we','negate','compare','affect','posemo','negemo','anx','anger',
                             'sad','social','family','friend','female','male','certain','health','achieve',
                             'reward','risk','work','leisure','home','death','swear','netspeak','extended_tweet']]
# for the datetime column keep the date only
Cognizant['created_at'] = Cognizant['created_at'].str.split(' ').str[0]
# convert the data type
Cognizant.created_at = pd.to_datetime(Cognizant.created_at, format='%Y-%m-%d')
# aggregate all columns at days except the text data concatenated but separated by '||'
Cognizant = Cognizant.groupby('created_at').agg(we=pd.NamedAgg(column='we', aggfunc=sum),
                                                           negate=pd.NamedAgg(column='negate', aggfunc=sum),
                                                           compare=pd.NamedAgg(column='compare', aggfunc=sum),
                                                           affect=pd.NamedAgg(column='affect', aggfunc=sum),
                                                           posemo=pd.NamedAgg(column='posemo', aggfunc=sum),
                                                           negemo=pd.NamedAgg(column='negemo', aggfunc=sum),
                                                           anx=pd.NamedAgg(column='anx', aggfunc=sum),
                                                           anger=pd.NamedAgg(column='anger', aggfunc=sum),
                                                           sad=pd.NamedAgg(column='sad', aggfunc=sum),
                                                           social=pd.NamedAgg(column='social', aggfunc=sum),
                                                           family=pd.NamedAgg(column='family', aggfunc=sum),
                                                           friend=pd.NamedAgg(column='friend', aggfunc=sum),
                                                           female=pd.NamedAgg(column='female', aggfunc=sum),
                                                           male=pd.NamedAgg(column='male', aggfunc=sum),
                                                           certain=pd.NamedAgg(column='certain', aggfunc=sum),
                                                           health=pd.NamedAgg(column='health', aggfunc=sum),
                                                           achieve=pd.NamedAgg(column='achieve', aggfunc=sum),
                                                           reward=pd.NamedAgg(column='reward', aggfunc=sum),
                                                           risk=pd.NamedAgg(column='risk', aggfunc=sum),
                                                           work=pd.NamedAgg(column='work', aggfunc=sum),
                                                           leisure=pd.NamedAgg(column='leisure', aggfunc=sum),
                                                           home=pd.NamedAgg(column='home', aggfunc=sum),
                                                           death=pd.NamedAgg(column='death', aggfunc=sum),
                                                           swear=pd.NamedAgg(column='swear', aggfunc=sum),
                                                           netspeak=pd.NamedAgg(column='netspeak', aggfunc=sum),
                                                           text_agg=pd.NamedAgg(column='extended_tweet', aggfunc=lambda x: x.str.cat(sep='||')))
# lowercase the text data
Cognizant['text_agg'] = Cognizant['text_agg'].str.lower()
# create the dummy variable for text data
Cognizant['dummy_key'] = Cognizant['text_agg'].apply(lambda x: 1 if ('covid' in x)|('pandemic' in x)|('coronavirus' in x) else 0)
Cognizant

,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak,text_agg,dummy_key
created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-05-28,0.00,8.40,12.24,16.79,4.55,12.24,4.55,0.0,3.85,30.40,0.00,0.0,0.00,4.55,3.85,13.86,22.70,4.55,3.85,49.71,5.00,0.00,0.0,0.0,37.26,🎉congratulations to @angiodynamics which is ce...,0
2019-05-29,0.00,4.35,12.35,8.35,8.35,0.00,0.00,0.0,0.00,9.09,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.83,0.00,0.00,0.0,0.0,12.90,🎧listen: @evanharvey99 and @yale lecturer in s...,0
2019-05-30,4.76,0.00,12.00,30.70,25.94,4.76,4.76,0.0,0.00,67.67,0.00,0.0,0.00,0.00,5.56,13.22,35.11,15.76,14.28,55.48,13.91,4.00,0.0,0.0,40.61,⚡️conmed corporation is innovating surgical te...,0
2019-05-31,4.00,0.00,9.56,5.56,5.56,0.00,0.00,0.0,0.00,9.56,0.00,0.0,0.00,0.00,0.00,5.56,11.11,5.56,4.00,16.70,8.35,0.00,0.0,0.0,17.91,we need to eliminate the order protection rule...,0
2019-06-01,0.00,8.00,0.00,8.00,4.00,4.00,0.00,0.0,4.00,8.55,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.64,0.00,0.00,0.0,0.0,12.55,"1,000+ members of the ir and capital markets e...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-05,19.14,0.00,11.14,37.94,37.94,0.00,0.00,0.0,0.00,94.51,17.18,0.0,0.00,3.57,13.61,0.00,16.76,17.11,4.00,71.69,17.18,17.18,0.0,0.0,50.76,defiance etfs announced the launch of the defi...,0
2020-08-06,7.41,3.70,4.35,20.77,17.07,3.70,0.00,0.0,3.70,50.54,4.55,0.0,0.00,4.00,0.00,3.70,8.00,12.68,4.00,68.44,8.90,4.55,0.0,0.0,34.02,.@nasdaq chief economist @phil_mackintosh disc...,0
2020-08-07,12.35,0.00,12.00,22.46,13.70,8.76,4.76,4.0,0.00,78.53,9.35,0.0,0.00,4.17,13.76,0.00,21.24,26.16,0.00,47.98,22.61,9.35,0.0,0.0,39.76,today! join subject matter experts from nasdaq...,1


In [ ]:
# plot the trends
x_axis = Cognizant.index
fig = go.Figure()
annotations = []

for i in range(0,25,1):
  y = Cognizant.iloc[:,i].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[:], y=y,
                    mode='lines',
                    name=Cognizant.columns[i]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LIWC Output for Cognizant',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations, 
                  autosize=False,
                  width=1200,
                  height=500,
                  margin=dict(l=10, r=10, b=50, t=50, pad=4))
fig.show()

# salesforce

In [ ]:
# subset data for the salesforce company
salesforce = data['screen_name'] == 'salesforce'
salesforce = data[salesforce]
# select the datetime, the targeted LIWC output variables, and the integral text data
salesforce = salesforce[['created_at','we','negate','compare','affect','posemo','negemo','anx','anger',
                             'sad','social','family','friend','female','male','certain','health','achieve',
                             'reward','risk','work','leisure','home','death','swear','netspeak','extended_tweet']]
# for the datetime column keep the date only
salesforce['created_at'] = salesforce['created_at'].str.split(' ').str[0]
# convert the data type
salesforce.created_at = pd.to_datetime(salesforce.created_at, format='%Y-%m-%d')
# aggregate all columns at days except the text data concatenated but separated by '||'
salesforce = salesforce.groupby('created_at').agg(we=pd.NamedAgg(column='we', aggfunc=sum),
                                                           negate=pd.NamedAgg(column='negate', aggfunc=sum),
                                                           compare=pd.NamedAgg(column='compare', aggfunc=sum),
                                                           affect=pd.NamedAgg(column='affect', aggfunc=sum),
                                                           posemo=pd.NamedAgg(column='posemo', aggfunc=sum),
                                                           negemo=pd.NamedAgg(column='negemo', aggfunc=sum),
                                                           anx=pd.NamedAgg(column='anx', aggfunc=sum),
                                                           anger=pd.NamedAgg(column='anger', aggfunc=sum),
                                                           sad=pd.NamedAgg(column='sad', aggfunc=sum),
                                                           social=pd.NamedAgg(column='social', aggfunc=sum),
                                                           family=pd.NamedAgg(column='family', aggfunc=sum),
                                                           friend=pd.NamedAgg(column='friend', aggfunc=sum),
                                                           female=pd.NamedAgg(column='female', aggfunc=sum),
                                                           male=pd.NamedAgg(column='male', aggfunc=sum),
                                                           certain=pd.NamedAgg(column='certain', aggfunc=sum),
                                                           health=pd.NamedAgg(column='health', aggfunc=sum),
                                                           achieve=pd.NamedAgg(column='achieve', aggfunc=sum),
                                                           reward=pd.NamedAgg(column='reward', aggfunc=sum),
                                                           risk=pd.NamedAgg(column='risk', aggfunc=sum),
                                                           work=pd.NamedAgg(column='work', aggfunc=sum),
                                                           leisure=pd.NamedAgg(column='leisure', aggfunc=sum),
                                                           home=pd.NamedAgg(column='home', aggfunc=sum),
                                                           death=pd.NamedAgg(column='death', aggfunc=sum),
                                                           swear=pd.NamedAgg(column='swear', aggfunc=sum),
                                                           netspeak=pd.NamedAgg(column='netspeak', aggfunc=sum),
                                                           text_agg=pd.NamedAgg(column='extended_tweet', aggfunc=lambda x: x.str.cat(sep='||')))
# lowercase the text data
salesforce['text_agg'] = salesforce['text_agg'].str.lower()
# create the dummy variable for text data
salesforce['dummy_key'] = salesforce['text_agg'].apply(lambda x: 1 if ('covid' in x)|('pandemic' in x)|('coronavirus' in x) else 0)
salesforce

,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak,text_agg,dummy_key
created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-05-28,0.00,8.40,12.24,16.79,4.55,12.24,4.55,0.0,3.85,30.40,0.00,0.0,0.00,4.55,3.85,13.86,22.70,4.55,3.85,49.71,5.00,0.00,0.0,0.0,37.26,🎉congratulations to @angiodynamics which is ce...,0
2019-05-29,0.00,4.35,12.35,8.35,8.35,0.00,0.00,0.0,0.00,9.09,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.83,0.00,0.00,0.0,0.0,12.90,🎧listen: @evanharvey99 and @yale lecturer in s...,0
2019-05-30,4.76,0.00,12.00,30.70,25.94,4.76,4.76,0.0,0.00,67.67,0.00,0.0,0.00,0.00,5.56,13.22,35.11,15.76,14.28,55.48,13.91,4.00,0.0,0.0,40.61,⚡️conmed corporation is innovating surgical te...,0
2019-05-31,4.00,0.00,9.56,5.56,5.56,0.00,0.00,0.0,0.00,9.56,0.00,0.0,0.00,0.00,0.00,5.56,11.11,5.56,4.00,16.70,8.35,0.00,0.0,0.0,17.91,we need to eliminate the order protection rule...,0
2019-06-01,0.00,8.00,0.00,8.00,4.00,4.00,0.00,0.0,4.00,8.55,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.64,0.00,0.00,0.0,0.0,12.55,"1,000+ members of the ir and capital markets e...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-05,19.14,0.00,11.14,37.94,37.94,0.00,0.00,0.0,0.00,94.51,17.18,0.0,0.00,3.57,13.61,0.00,16.76,17.11,4.00,71.69,17.18,17.18,0.0,0.0,50.76,defiance etfs announced the launch of the defi...,0
2020-08-06,7.41,3.70,4.35,20.77,17.07,3.70,0.00,0.0,3.70,50.54,4.55,0.0,0.00,4.00,0.00,3.70,8.00,12.68,4.00,68.44,8.90,4.55,0.0,0.0,34.02,.@nasdaq chief economist @phil_mackintosh disc...,0
2020-08-07,12.35,0.00,12.00,22.46,13.70,8.76,4.76,4.0,0.00,78.53,9.35,0.0,0.00,4.17,13.76,0.00,21.24,26.16,0.00,47.98,22.61,9.35,0.0,0.0,39.76,today! join subject matter experts from nasdaq...,1


In [ ]:
# plot the trends
x_axis = salesforce.index
fig = go.Figure()
annotations = []

for i in range(0,25,1):
  y = salesforce.iloc[:,i].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[:], y=y,
                    mode='lines',
                    name=salesforce.columns[i]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LIWC Output for salesforce',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations, 
                  autosize=False,
                  width=1200,
                  height=500,
                  margin=dict(l=10, r=10, b=50, t=50, pad=4))
fig.show()

# SPGlobal

In [ ]:
# subset data for the SPGlobal company
SPGlobal = data['screen_name'] == 'SPGlobal'
SPGlobal = data[SPGlobal]
# select the datetime, the targeted LIWC output variables, and the integral text data
SPGlobal = SPGlobal[['created_at','we','negate','compare','affect','posemo','negemo','anx','anger',
                             'sad','social','family','friend','female','male','certain','health','achieve',
                             'reward','risk','work','leisure','home','death','swear','netspeak','extended_tweet']]
# for the datetime column keep the date only
SPGlobal['created_at'] = SPGlobal['created_at'].str.split(' ').str[0]
# convert the data type
SPGlobal.created_at = pd.to_datetime(SPGlobal.created_at, format='%Y-%m-%d')
# aggregate all columns at days except the text data concatenated but separated by '||'
SPGlobal = SPGlobal.groupby('created_at').agg(we=pd.NamedAgg(column='we', aggfunc=sum),
                                                           negate=pd.NamedAgg(column='negate', aggfunc=sum),
                                                           compare=pd.NamedAgg(column='compare', aggfunc=sum),
                                                           affect=pd.NamedAgg(column='affect', aggfunc=sum),
                                                           posemo=pd.NamedAgg(column='posemo', aggfunc=sum),
                                                           negemo=pd.NamedAgg(column='negemo', aggfunc=sum),
                                                           anx=pd.NamedAgg(column='anx', aggfunc=sum),
                                                           anger=pd.NamedAgg(column='anger', aggfunc=sum),
                                                           sad=pd.NamedAgg(column='sad', aggfunc=sum),
                                                           social=pd.NamedAgg(column='social', aggfunc=sum),
                                                           family=pd.NamedAgg(column='family', aggfunc=sum),
                                                           friend=pd.NamedAgg(column='friend', aggfunc=sum),
                                                           female=pd.NamedAgg(column='female', aggfunc=sum),
                                                           male=pd.NamedAgg(column='male', aggfunc=sum),
                                                           certain=pd.NamedAgg(column='certain', aggfunc=sum),
                                                           health=pd.NamedAgg(column='health', aggfunc=sum),
                                                           achieve=pd.NamedAgg(column='achieve', aggfunc=sum),
                                                           reward=pd.NamedAgg(column='reward', aggfunc=sum),
                                                           risk=pd.NamedAgg(column='risk', aggfunc=sum),
                                                           work=pd.NamedAgg(column='work', aggfunc=sum),
                                                           leisure=pd.NamedAgg(column='leisure', aggfunc=sum),
                                                           home=pd.NamedAgg(column='home', aggfunc=sum),
                                                           death=pd.NamedAgg(column='death', aggfunc=sum),
                                                           swear=pd.NamedAgg(column='swear', aggfunc=sum),
                                                           netspeak=pd.NamedAgg(column='netspeak', aggfunc=sum),
                                                           text_agg=pd.NamedAgg(column='extended_tweet', aggfunc=lambda x: x.str.cat(sep='||')))
# lowercase the text data
SPGlobal['text_agg'] = SPGlobal['text_agg'].str.lower()
# create the dummy variable for text data
SPGlobal['dummy_key'] = SPGlobal['text_agg'].apply(lambda x: 1 if ('covid' in x)|('pandemic' in x)|('coronavirus' in x) else 0)
SPGlobal

,we,negate,compare,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,certain,health,achieve,reward,risk,work,leisure,home,death,swear,netspeak,text_agg,dummy_key
created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-05-28,0.00,8.40,12.24,16.79,4.55,12.24,4.55,0.0,3.85,30.40,0.00,0.0,0.00,4.55,3.85,13.86,22.70,4.55,3.85,49.71,5.00,0.00,0.0,0.0,37.26,🎉congratulations to @angiodynamics which is ce...,0
2019-05-29,0.00,4.35,12.35,8.35,8.35,0.00,0.00,0.0,0.00,9.09,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.83,0.00,0.00,0.0,0.0,12.90,🎧listen: @evanharvey99 and @yale lecturer in s...,0
2019-05-30,4.76,0.00,12.00,30.70,25.94,4.76,4.76,0.0,0.00,67.67,0.00,0.0,0.00,0.00,5.56,13.22,35.11,15.76,14.28,55.48,13.91,4.00,0.0,0.0,40.61,⚡️conmed corporation is innovating surgical te...,0
2019-05-31,4.00,0.00,9.56,5.56,5.56,0.00,0.00,0.0,0.00,9.56,0.00,0.0,0.00,0.00,0.00,5.56,11.11,5.56,4.00,16.70,8.35,0.00,0.0,0.0,17.91,we need to eliminate the order protection rule...,0
2019-06-01,0.00,8.00,0.00,8.00,4.00,4.00,0.00,0.0,4.00,8.55,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.64,0.00,0.00,0.0,0.0,12.55,"1,000+ members of the ir and capital markets e...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-05,19.14,0.00,11.14,37.94,37.94,0.00,0.00,0.0,0.00,94.51,17.18,0.0,0.00,3.57,13.61,0.00,16.76,17.11,4.00,71.69,17.18,17.18,0.0,0.0,50.76,defiance etfs announced the launch of the defi...,0
2020-08-06,7.41,3.70,4.35,20.77,17.07,3.70,0.00,0.0,3.70,50.54,4.55,0.0,0.00,4.00,0.00,3.70,8.00,12.68,4.00,68.44,8.90,4.55,0.0,0.0,34.02,.@nasdaq chief economist @phil_mackintosh disc...,0
2020-08-07,12.35,0.00,12.00,22.46,13.70,8.76,4.76,4.0,0.00,78.53,9.35,0.0,0.00,4.17,13.76,0.00,21.24,26.16,0.00,47.98,22.61,9.35,0.0,0.0,39.76,today! join subject matter experts from nasdaq...,1


In [ ]:
# plot the trends
x_axis = SPGlobal.index
fig = go.Figure()
annotations = []

for i in range(0,25,1):
  y = SPGlobal.iloc[:,i].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[:], y=y,
                    mode='lines',
                    name=SPGlobal.columns[i]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LIWC Output for SPGlobal',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations, 
                  autosize=False,
                  width=1200,
                  height=500,
                  margin=dict(l=10, r=10, b=50, t=50, pad=4))
fig.show()